1. Add the "OwnerParcel" layer from the parcels GDB for the city
2. Add the zoning layer
Zoning from https://maps.cityofwhitefish.org/server/rest/services/kiosk_zones/MapServer/0

In [ ]:
import os
import shutil

def index_of(city_name):
    i = next((index for (index, d) in enumerate(cities) if d["name"] == city_name), None)
    return i

cities = [
    {
        'name': 'Billings',
        'zoning_field': 'ZONE_CODE',
        'county': 'Yellowstone',
        'residential_zones': ['N1', 'N2', 'N3', 'NX1', 'NX2', 'NX3', 'RMH']
    },
    {
        'name': 'Bozeman',
        'zoning_field': 'ZONING',
        'county': 'Gallatin',
        'residential_zones': ['R-S', 'R-1', 'R-2', 'R-3', 
        'R-4', 'R-5', 'R-O', 'RMH']
    },
    {
        'name': 'Helena',
        'zoning_field': 'ZoneCode',
        'county': 'LewisClark',
        'residential_zones': ['R1', 'R2', 'R3', 'R4', 'RO', 'RU']
    },
    {
        'name': 'Kalispell',
        'zoning_field': 'ZONE_USE',
        'county': 'Flathead',
        'residential_zones': ['R-1', 'R-2', 'R-3', 
                               'R-4', 'R-5', 'RA-1', 'RA-2']
    },
    {
        'name': 'Missoula',
        'zoning_field': 'Zoning',
        'county': 'Missoula',
        'residential_zones': ['R215', 'R80', 'R40', 'R20', 'RT10', 'R8', 
        'R5.4', 'RT5.4', 'R3', 'RT2.7', 'RM2.7', 'RM2', 'RM1.5', 
        'RM1', 'RM0.5', 'RMH']
    },
    {
        'name': 'Whitefish',
        'zoning_field': 'zone_use',
        'county': 'Flathead',
        'residential_zones': ['WA', 'WCR', 'WSR', 'WER', 'WLR', 'WR-1', 
                             'WR-2', 'WR-3', 'WR-4', 'WRR-1', 'WRR-2', 'WBMRR']
    }
]

for city in cities:
    city['gdb'] = os.path.join('cities', city['name'] + '.gdb')
    city['layer'] = os.path.join(city['gdb'], 'Parcels_' + city['name'])

In [ ]:
# Create city GDBs
# Copy in the zoning layer
# Rename the appropriate field in the zoning layer to 'zoning'

for city in cities: 
    source_zoning = os.path.join('shp', 'zoning.gdb', city['name'].lower())
    
    # create GDB if not exists
    if not os.path.isdir(city['gdb']):
        arcpy.management.CreateFileGDB('cities', city['name'])

    arcpy.conversion.FeatureClassToFeatureClass(
        source_zoning,
        city['gdb'],
        "Zoning"
    )
    
    # rename zoning code field to zoning
    
    arcpy.management.AlterField(
        os.path.join(city['gdb'], 'Zoning'),
        city['zoning_field'],
        'zoning_temp'
    )
    
    arcpy.management.AlterField(
        os.path.join(city['gdb'], 'Zoning'),
        'zoning_temp',
        'zoning', 
        'Zoning Code'
    )
    
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    mp = aprx.listMaps("Map")[0]
    for layer in mp.listLayers():
        if layer.name in ["Zoning"]:
            mp.removeLayer(layer)
        

In [ ]:
# Join the parcels and the zoning

for city in cities:
    source_parcels = r'shp\{0}_GDB\{0}_Parcels.gdb\OwnerParcel'.format(city['county'])
    
    where_clause = """
    PARCELID IS NOT NULL 
    AND PropType NOT IN (
            '',
            'VR - Vacant Land Rural', 
            'CA - Centrally Assessed', 
            'VU - Vacant Land Urban',
            'FARM_U - Farmstead - Urban', 
            'NVS - Non-Valued with Specials', 
            'RV_PARK - RV Park', 
            'GRAVEL - Gravel Pit', 
            'GOLF - Golf Course',
            'EP_PART - Partial Exempt Property', 
            'CN - Centrally Assessed Non-Valued Property', 
            'NV - Non-Valued Property', 'FARM_R - Farmstead - Rural', 
            'VAC_U - Vacant Land - Urban', 
            'EP - Exempt Property', 'VAC_R - Vacant Land - Rural'
        )
    """
    arcpy.management.MakeFeatureLayer(
        source_parcels, 
        'cadastral_parcels', 
        where_clause
    )
    
    where_clause =  "zoning IN ('" + "', '".join(city['residential_zones']) + "')"
    arcpy.management.MakeFeatureLayer(
        os.path.join(city['gdb'], 'Zoning'),
        'zoning', 
        where_clause
    )
    
    arcpy.analysis.SpatialJoin(
        'cadastral_parcels', 
        'zoning', 
        city['layer'], 
        "JOIN_ONE_TO_ONE", 
        "KEEP_COMMON", 
        None, 
        "HAVE_THEIR_CENTER_IN"
    )
    
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    mp = aprx.listMaps("Map")[0]
    for layer in mp.listLayers():
        if layer.name in ['parcels', 'zoning', 'cadastral_parcels']:
            mp.removeLayer(layer)

In [ ]:
# calculate lot sizes
for city in cities:

    # calculate lot areas in SF and acres
    arcpy.management.CalculateGeometryAttributes(
        city['layer'], 
        "lot_size AREA", 
        '', 
        "SQUARE_FEET_US", 
        None, 
        "SAME_AS_INPUT"
    )
    arcpy.management.CalculateField(
        city['layer'], 
        "lot_size", 
        "round(!lot_size!)", 
        "PYTHON3", 
        None, 
        "SHORT"
    )

    # calculate lot areas in SF and acres
    arcpy.management.CalculateField(
        city['layer'], 
        "lot_acres", 
        "round(!lot_size! / 43560, 3)", 
        "PYTHON3", 
        None, 
        "DOUBLE"
    )

In [ ]:
# add city name field
for city in cities:
    arcpy.management.CalculateField(
        city['layer'], 
        "city", 
        '"' + city['name'] + '"'
    )

In [ ]:
# add calulate lot widths

for city in cities:
    
    arcpy.management.Project(
        city['layer'], 
        os.path.join(city['gdb'], 'parcels_ft'),
        'PROJCS["NAD_1983_StatePlane_Montana_FIPS_2500_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Foot_US",0.3048006096012192]]', "'WGS_1984_(ITRF08)_To_NAD_1983_2011 + WGS_1984_(ITRF00)_To_NAD_1983'", 'PROJCS["NAD_1983_2011_StatePlane_Montana_FIPS_2500",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",600000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Meter",1.0]]'
    )
    
    arcpy.management.MinimumBoundingGeometry(
        os.path.join(city['gdb'], 'parcels_ft'),
        r'memory\parcels_mbb', 
        "RECTANGLE_BY_WIDTH", 
        "NONE", 
        None, 
        "MBG_FIELDS"
    )
    
    arcpy.management.JoinField(
        city['layer'], 
        'PARCELID', 
        r'memory\parcels_mbb', 
        'PARCELID', 
        'MBG_Width;MBG_Length'
    )
    
    arcpy.management.CalculateField(
        city['layer'], 
        "lot_width", 
        "!MBG_Width!", 
        "PYTHON3", 
        None, 
        "SHORT"
    )

    arcpy.management.Delete(r'memory\parcels_mbb')

In [ ]:
arcpy.management.SelectLayerByAttribute(
    'Parcels_' + 'Billings', 
    "NEW_SELECTION", 
    "lot_acres >= 160 Or lot_size < 250"
)


In [ ]:
# remove parcels less than 500 SF and greater than 160 acres
# make sure these are layers in contents pane
for city in cities:
    
    arcpy.management.MakeFeatureLayer(
        city['layer'],
        'parcels'
    )
    
    arcpy.management.SelectLayerByAttribute(
        'parcels', 
        "NEW_SELECTION", 
        "lot_acres >= 160 Or lot_size < 250"
    )
    
    if int(arcpy.management.GetCount('parcels')[0]) < 500:
        arcpy.management.DeleteFeatures('parcels')
    else:
        print('too many features selected ' + city['name'])
    
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    mp = aprx.listMaps("Map")[0]
    for layer in mp.listLayers():
        if layer.name in ['parcels', 'Parcels_' + city['name']]:
            mp.removeLayer(layer)

## Calcualte zoning facts/figures

## Whitefish

## Kalispell

## Missoula

## Helena

## Billings

## Bozeman

## Save final results

In [ ]:
codeblock = """
def units(zone, size, width):
    return max(1, min(zone, size, width))
"""
for city in cities:
    arcpy.management.CalculateField(
        city['layer'], 
        "units_allowed", 
        "units(!zoning_allows!, !lot_size_allows!, !lot_width_allows!)",
        "PYTHON3",
        codeblock,
        "SHORT"
    )

In [ ]:
for city in cities:
    field_spec = 'city "City" true true false 512 Text 0 0,First,#,{0},city,0,512;zoning "Zoning" true true false 512 Text 0 0,First,#,{0},zoning,0,512;lot_size "Lot Size" true true false 8 Double 0 0,First,#,{0},lot_size,-1,-1;lot_acres "Lot Size (ac.)" true true false 8 Double 0 0,First,#,{0},lot_acres,-1,-1;lot_width "Lot Width" true true false 2 Short 0 0,First,#,{0},lot_width,-1,-1;zoning_allows "Units Allowed by Zoning" true true false 2 Short 0 0,First,#,{0},zoning_allows,-1,-1;lot_size_allows "Units Allowed by Lot Size" true true false 2 Short 0 0,First,#,{0},lot_size_allows,-1,-1;lot_width_allows "Units Allowed by Lot Width" true true false 2 Short 0 0,First,#,{0},lot_width_allows,-1,-1;units_allowed "Units Allowed" true true false 2 Short 0 0,First,#,{0},units_allowed,-1,-1;category "Residential Category" true true false 512 Text 0 0,First,#,{0},category,0,512'.format('Parcels_' + city['name'])
    arcpy.conversion.FeatureClassToFeatureClass(
        city['layer'], 
        city['gdb'], 
        city['name'].lower() + "_residential_parcels", 
        '', 
        field_spec,
        ''
    )

## Merge Layers

In [ ]:
arcpy.management.Merge(
    r"cities\Billings.gdb\billings_residential_parcels;cities\Helena.gdb\helena_residential_parcels;cities\Whitefish.gdb\whitefish_residential_parcels;cities\Bozeman.gdb\bozeman_residential_parcels;cities\Kalispell.gdb\kalispell_residential_parcels;cities\Missoula.gdb\missoula_residential_parcels", r"zoning_atlas.gdb\parcels", 
    r'city "City" true true false 40 Text 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,city,0,512,cities\Helena.gdb\helena_residential_parcels,city,0,512,cities\Whitefish.gdb\whitefish_residential_parcels,city,0,512,cities\Bozeman.gdb\bozeman_residential_parcels,city,0,512,cities\Kalispell.gdb\kalispell_residential_parcels,city,0,512,cities\Missoula.gdb\missoula_residential_parcels,city,0,512;zoning "Zoning" true true false 512 Text 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,zoning,0,512,cities\Helena.gdb\helena_residential_parcels,zoning,0,512,cities\Whitefish.gdb\whitefish_residential_parcels,zoning,0,512,cities\Bozeman.gdb\bozeman_residential_parcels,zoning,0,512,cities\Kalispell.gdb\kalispell_residential_parcels,zoning,0,512,cities\Missoula.gdb\missoula_residential_parcels,zoning,0,512;lot_size "Lot Size" true true false 8 Double 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,lot_size,-1,-1,cities\Helena.gdb\helena_residential_parcels,lot_size,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,lot_size,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,lot_size,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,lot_size,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,lot_size,-1,-1;lot_acers "Lot Size (ac.)" true true false 8 Double 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,lot_acers,-1,-1,cities\Helena.gdb\helena_residential_parcels,lot_acers,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,lot_acers,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,lot_acers,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,lot_acers,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,lot_acers,-1,-1;lot_width "Lot Width" true true false 2 Short 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,lot_width,-1,-1,cities\Helena.gdb\helena_residential_parcels,lot_width,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,lot_width,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,lot_width,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,lot_width,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,lot_width,-1,-1;zoning_allows "Units Allowed by Zoning" true true false 2 Short 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,zoning_allows,-1,-1,cities\Helena.gdb\helena_residential_parcels,zoning_allows,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,zoning_allows,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,zoning_allows,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,zoning_allows,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,zoning_allows,-1,-1;lot_size_allows "Units Allowed by Lot Size" true true false 2 Short 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,lot_size_allows,-1,-1,cities\Helena.gdb\helena_residential_parcels,lot_size_allows,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,lot_size_allows,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,lot_size_allows,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,lot_size_allows,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,lot_size_allows,-1,-1;lot_width_allows "Units Allowed by Lot Width" true true false 2 Short 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,lot_width_allows,-1,-1,cities\Helena.gdb\helena_residential_parcels,lot_width_allows,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,lot_width_allows,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,lot_width_allows,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,lot_width_allows,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,lot_width_allows,-1,-1;units_allowed "Units Allowed" true true false 2 Short 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,units_allowed,-1,-1,cities\Helena.gdb\helena_residential_parcels,units_allowed,-1,-1,cities\Whitefish.gdb\whitefish_residential_parcels,units_allowed,-1,-1,cities\Bozeman.gdb\bozeman_residential_parcels,units_allowed,-1,-1,cities\Kalispell.gdb\kalispell_residential_parcels,units_allowed,-1,-1,cities\Missoula.gdb\missoula_residential_parcels,units_allowed,-1,-1;category "Residential Category" true true false 512 Text 0 0,First,#,cities\Billings.gdb\billings_residential_parcels,category,0,512,cities\Helena.gdb\helena_residential_parcels,category,0,512,cities\Whitefish.gdb\whitefish_residential_parcels,category,0,512,cities\Bozeman.gdb\bozeman_residential_parcels,category,0,512,cities\Kalispell.gdb\kalispell_residential_parcels,category,0,512,cities\Missoula.gdb\missoula_residential_parcels,category,0,512', 
    "NO_SOURCE_INFO"
)

In [ ]:
arcpy.management.SelectLayerByAttribute(
    "parcels", 
    "NEW_SELECTION", 
    "category <> 'Single-Family' And units_allowed <= 1", None)

arcpy.management.CalculateField(
    "parcels", 
    "category", 
    '"De Facto Single-Family"', 
    "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")



In [ ]:
arcpy.management.SelectLayerByAttribute(
    "parcels", 
    "NEW_SELECTION", 
    "category = 'Single-Family'", None)
arcpy.management.CalculateField(
    "parcels", 
    "category", 
    '"Single-Family Zoning"', 
    "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

## Combine Zoning Code Layers

In [1]:
arcpy.management.Merge(
r"cities\Billings.gdb\Zoning;cities\Bozeman.gdb\Zoning;cities\Helena.gdb\Zoning;cities\Kalispell.gdb\Zoning;cities\Missoula.gdb\Zoning;cities\Whitefish.gdb\Zoning", 
    r"story_map_2.gdb\zoning_codes_combined", 
    r'zoning "Zoning Code" true true false 10 Text 0 0,First,#,cities\Billings.gdb\Zoning,zoning,0,10,cities\Bozeman.gdb\Zoning,zoning,0,5,cities\Helena.gdb\Zoning,zoning,0,5,cities\Kalispell.gdb\Zoning,zoning,0,254,cities\Missoula.gdb\Zoning,zoning,0,80,cities\Whitefish.gdb\Zoning,zoning,0,30;ZONE_NAME "ZONE_NAME" true true false 75 Text 0 0,First,#,cities\Billings.gdb\Zoning,ZONE_NAME,0,75,cities\Helena.gdb\Zoning,ZONE_NAME,0,70;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,cities\Billings.gdb\Zoning,Shape_Length,-1,-1,cities\Bozeman.gdb\Zoning,Shape_Length,-1,-1,cities\Helena.gdb\Zoning,Shape_Length,-1,-1,cities\Kalispell.gdb\Zoning,Shape_Length,-1,-1,cities\Missoula.gdb\Zoning,Shape_Length,-1,-1,cities\Whitefish.gdb\Zoning,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,cities\Billings.gdb\Zoning,Shape_Area,-1,-1,cities\Bozeman.gdb\Zoning,Shape_Area,-1,-1,cities\Helena.gdb\Zoning,Shape_Area,-1,-1,cities\Kalispell.gdb\Zoning,Shape_Area,-1,-1,cities\Missoula.gdb\Zoning,Shape_Area,-1,-1,cities\Whitefish.gdb\Zoning,Shape_Area,-1,-1;created_user "created_user" true true false 255 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,created_user,0,255,cities\Helena.gdb\Zoning,created_user,0,255,cities\Missoula.gdb\Zoning,created_user,0,255;created_date "created_date" true true false 8 Date 0 0,First,#,cities\Bozeman.gdb\Zoning,created_date,-1,-1,cities\Helena.gdb\Zoning,created_date,-1,-1,cities\Missoula.gdb\Zoning,created_date,-1,-1;last_edited_user "last_edited_user" true true false 255 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,last_edited_user,0,255,cities\Helena.gdb\Zoning,last_edited_user,0,255,cities\Missoula.gdb\Zoning,last_edited_user,0,255;last_edited_date "last_edited_date" true true false 8 Date 0 0,First,#,cities\Bozeman.gdb\Zoning,last_edited_date,-1,-1,cities\Helena.gdb\Zoning,last_edited_date,-1,-1,cities\Missoula.gdb\Zoning,last_edited_date,-1,-1;STR_TYPE_1 "STR TYPE 1" true true false 10 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,STR_TYPE_1,0,10;STR_TYPE_2 "STR TYPE 2" true true false 10 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,STR_TYPE_2,0,10;STR_TYPE_3 "STR TYPE 3" true true false 10 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,STR_TYPE_3,0,10;Muni_Hyperlink "Muni_Hyperlink" true true false 300 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,Muni_Hyperlink,0,300;Muni_Hyperlink_Uses "Muni_Hyperlink_Uses" true true false 250 Text 0 0,First,#,cities\Bozeman.gdb\Zoning,Muni_Hyperlink_Uses,0,250;ZONE_ "ZONE_" true true false 10 Text 0 0,First,#,cities\Helena.gdb\Zoning,ZONE_,0,10;PUD_NAME "PUD_NAME" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,PUD_NAME,0,254;PUD_NUM "PUD_NUM" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,PUD_NUM,0,254;USE_DESCR "USE_DESCR" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,USE_DESCR,0,254;ZONE_DIST "ZONE_DIST" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,ZONE_DIST,0,254;PUD_ORD "PUD_ORD" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,PUD_ORD,0,254;ORD_DATE "ORD_DATE" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,ORD_DATE,0,254;MINLOT_AR "MINLOT_AR" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,MINLOT_AR,0,254;MINLOT_WTH "MINLOT_WTH" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,MINLOT_WTH,0,254;FRONT_SB "FRONT_SB" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,FRONT_SB,0,254;SIDE_SB "SIDE_SB" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,SIDE_SB,0,254;REAR_SB "REAR_SB" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,REAR_SB,0,254;CORNER_SB "CORNER_SB" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,CORNER_SB,0,254;MAXBLDG_HT "MAXBLDG_HT" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,MAXBLDG_HT,0,254;MAX_LOTCOV "MAX_LOTCOV" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,MAX_LOTCOV,0,254;F_FENCE_HT "F_FENCE_HT" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,F_FENCE_HT,0,254;S_FENCE_HT "S_FENCE_HT" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,S_FENCE_HT,0,254;R_FENCE_HT "R_FENCE_HT" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,R_FENCE_HT,0,254;MEMO_ "MEMO_" true true false 254 Text 0 0,First,#,cities\Kalispell.gdb\Zoning,MEMO_,0,254;Base "Base" true true false 80 Text 0 0,First,#,cities\Missoula.gdb\Zoning,Base,0,80;Overlay "Overlay" true true false 80 Text 0 0,First,#,cities\Missoula.gdb\Zoning,Overlay,0,80;OrdinanceNumber "Ordinance Number" true true false 30 Text 0 0,First,#,cities\Missoula.gdb\Zoning,OrdinanceNumber,0,30;OrdinanceDate "Ordinance Date" true true false 8 Date 0 0,First,#,cities\Missoula.gdb\Zoning,OrdinanceDate,-1,-1;ResolutionNumber "Resolution Number" true true false 30 Text 0 0,First,#,cities\Missoula.gdb\Zoning,ResolutionNumber,0,30;ResolutionDate "Resolution Date" true true false 8 Date 0 0,First,#,cities\Missoula.gdb\Zoning,ResolutionDate,-1,-1;Acres "Area (Acres)" true true false 8 Double 0 0,First,#,cities\Missoula.gdb\Zoning,Acres,-1,-1;GlobalID "GlobalID" false false false 38 GlobalID 0 0,First,#,cities\Missoula.gdb\Zoning,GlobalID,-1,-1;ordinance "ordinance" true true false 50 Text 0 0,First,#,cities\Whitefish.gdb\Zoning,ordinance,0,50;proj_name "proj_name" true true false 66 Text 0 0,First,#,cities\Whitefish.gdb\Zoning,proj_name,0,66;zone_base "zone_base" true true false 33 Text 0 0,First,#,cities\Whitefish.gdb\Zoning,zone_base,0,33;zone_pud "zone_pud" true true false 8 Double 0 0,First,#,cities\Whitefish.gdb\Zoning,zone_pud,-1,-1;leglabel "leglabel" true true false 66 Text 0 0,First,#,cities\Whitefish.gdb\Zoning,leglabel,0,66;Date_added "Date_added" true true false 8 Date 0 0,First,#,cities\Whitefish.gdb\Zoning,Date_added,-1,-1;Short_Term_Rental "Short_Term_Rental" true true false 2 Short 0 0,First,#,cities\Whitefish.gdb\Zoning,Short_Term_Rental,-1,-1;CodeURL "Code URL" true true false 200 Text 0 0,First,#,cities\Whitefish.gdb\Zoning,CodeURL,0,200', "NO_SOURCE_INFO"
)

<Result 'story_map_2.gdb\\zoning_codes_combined'>